# Train over MNIST

### Trainning model with max pooling under LRP framework to save model's parameters

#### Imports

In [1]:
#python 2 and 3 comptibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#enables acces to parent folder
import os
import sys
sys.path.append("../..")

#other imports
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

#MNIST import
from tensorflow.examples.tutorials.mnist import input_data

#modules import
import modules.sequential32 as sequential32
import modules.linear32 as linear32
import modules.convolution32 as convolution32
import modules.maxpool32 as maxpool32
import modules.avgpool32 as avgpool32



#print of plots within notebook
%matplotlib inline

#### Trainning Parameters

In [2]:
#trainning iterations
max_steps = 1101 
batch_size = 50
learning_rate = 1e-4
dropout = 0.5
#where to save trainning checkpoints and tensorboard utils
summaries_dir="TB/mnist/"
#where to save parameters of the model
path_weights = 'weights/'

#### Execute if MNIST haven't been downladed
MNIST dataset weights approx. 12 MB. If it isn't previously downloaded on directory 'data/MNIST', it will be automatically downladed with next code cell.

In [3]:
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


#### Alternatively you can give a path to previously downloaded MNIST
[only execute if previous cell wasn't]

In [4]:
#MNIST path
from tensorflow.examples.tutorials.mnist import input_data
data_dir = '/home/tesla/Desktop/MNIST_data/data'
data = input_data.read_data_sets(data_dir, one_hot=True)

Extracting /home/tesla/Desktop/MNIST_data/data/train-images-idx3-ubyte.gz
Extracting /home/tesla/Desktop/MNIST_data/data/train-labels-idx1-ubyte.gz
Extracting /home/tesla/Desktop/MNIST_data/data/t10k-images-idx3-ubyte.gz
Extracting /home/tesla/Desktop/MNIST_data/data/t10k-labels-idx1-ubyte.gz


#### Print set lenghts

In [5]:
print("Number of sample in each set of MNIST:")
print("Training-set:\t\t{}".format(len(data.train.labels)))
print("Test-set:\t\t{}".format(len(data.test.labels)))

Number of sample in each set of MNIST:
Training-set:		55000
Test-set:		10000


#### Instantiatite network model

This consider a pretreined model. To be loaded, weights must be in a 'weights_MNIST_MaxPool/' folder, relative to this notebook.

In [6]:

def nn():
    return sequential32.Sequential([convolution32.Convolution(kernel_size=5, output_depth=32, input_depth=1,
                                   input_dim=28, act ='relu',
                                   stride_size=1, pad='SAME'),
                       maxpool32.MaxPool(),

                       convolution32.Convolution(kernel_size=5,output_depth=64, stride_size=1,
                                   act ='relu', pad='SAME'),
                       maxpool32.MaxPool(),
                       
                       linear32.Linear(1024, act ='relu'),

                       linear32.Linear(10, act ='linear')])  

#### Model I/O

In [7]:
#Dropout placeholder
keep_prob = tf.placeholder(tf.float32, name='keep-prob')

#Input place holders
with tf.name_scope('input'):
        #stetched image
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        #target label
        y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
  
        #reshape input as image tensor with an undifined batch size (-1)
        inp = tf.reshape(x, [-1, 28, 28, 1])

#Model instance
with tf.variable_scope('model'):
        #instanciate model
        net = nn()

        #feed-forward method and get score output. This iterates through layer in net object of class Sequiential
        score = net.forward(inp)

        #pass scores through softmax for network output ([0,1] probability)   
        y = tf.nn.softmax(score)

        #predicted classes
        y_pred_cls = tf.argmax(y, 1)

        #true classes
        y_true_cls = tf.argmax(y_, 1)
        
        ##Training function      
        trainer = net.fit(output=score, ground_truth=y_, opt_params=learning_rate)

Forward Pass ... 
------------------------------------------------- 
input:: [None, 28, 28, 1]
conv2d_1:: [None, 28, 28, 32]
maxpool_2:: [None, 14, 14, 32]
conv2d_3:: [None, 14, 14, 64]
maxpool_4:: [None, 7, 7, 64]
linear_5:: [None, 1024]
linear_6:: [None, 10]
softmax:: [None, 10]

------------------------------------------------- 


#### Accuracy measurement and create TensorBoard writers

In [9]:
with tf.name_scope('accuracy'):  
        #performance measures

        #compare predictions
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(y_pred_cls, y_true_cls)
        #get accuracy
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
accuracy_sum = tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge([accuracy_sum])

#### Init Tensorflow session, variables and tensorboard writers

In [10]:
config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())


train_writer = tf.summary.FileWriter(summaries_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(summaries_dir + '/test')

#### Functions to feed model with data
Generates proper functions to construct dictionary that will be feed to model graph. Define batch, labels and dropout.

There is the option to feed the model with a batch of MNIST, or feed it with a ten digits example.

__Remember:__ Images pixels are 0 centered in values [-1,1]

In [11]:
##Get MNIST Data to feed model. You can choose to feed a train batch or the whole test set 
def feed_dict(mnist, train, batch_s=None):
    if train:
        xs, ys = mnist.train.next_batch(batch_s)
        k = dropout
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return (2*xs)-1, ys, k

#### Function to Check accuracy of model

In [12]:
##Measure accuracy of the net
def print_test_accuracy(test_batch_size, mnist):
    data=mnist 
    data.test.cls = np.argmax(data.test.labels, axis=1)
    #data.validation.cls = np.argmax(data.test.labels, axis=1)
    #data.train.cls = np.argmax(data.train.labels, axis=1)
    # Numero de imagenes en test-set.
    num_test = len(data.test.images)

    # Crea arreglo para guardar clases predichas.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Calcular clases predichas.
    i = 0
    while i < num_test:
        
        j = min(i + test_batch_size, num_test)
        images = data.test.images[i:j, :]
        images = (2*images)-1
        labels = data.test.labels[i:j, :]
        dp=1
        feed_dict = {x: images,
                     y_: labels, keep_prob: dp}

        cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)
        i = j
    
    # Labels reales.
    cls_true = data.test.cls

    # Arreglo booleano de clasificaciones correctas.
    correct = (cls_true == cls_pred)
    
    #Numero de clasificaciones correctas.
    correct_sum = correct.sum()

    # Accuracy
    acc = float(correct_sum) / num_test
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))
    return acc, cls_true,  cls_pred

### Train model

train model for max_steps iterations and print accuracy at the end

In [16]:
#TRAIN
j=0
ep=55000/batch_size


for i in range(max_steps):
    d = feed_dict(data, True, batch_size)
    inp = {x:d[0], y_:d[1], keep_prob:d[2]}
    #inp = feed_dict(mnist, True, batch_size)
    
    sess.run(trainer.train, feed_dict=inp)

    if i % ep == 0:
        d_t = feed_dict(data, False, batch_size)
        inp_t = {x:d_t[0], y_:d_t[1], keep_prob:d_t[2]}
        summary, acc = sess.run([merged, accuracy], feed_dict=inp_t)
        test_writer.add_summary(summary, i)
        msg = "epoca: {0:>6}"
        print(msg.format(j+1))
        #print_test_accuracy(test_batch_size=batch_size, mnist=mnist)
        num_test = len(data.test.images)
        correct_sum = acc*num_test
        msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
        print(msg.format(acc, correct_sum, num_test))
        j+=1
        
        
    if i % (ep/10) == 0:
        inp = {x:d[0], y_:d[1], keep_prob:d_t[2]}
        summary, acc = sess.run([merged, accuracy], feed_dict=inp) #accuracy on batch
        train_writer.add_summary(summary, i)
        msg = "Iterations: {0:>6}, Training Accuracy: {1:>6.1%}"
        print(msg.format(i, acc))
        #Train_acc[int(i/(ep/10)),:]=[i,acc]

print_test_accuracy(test_batch_size=batch_size, mnist=data)

epoca:      1
Accuracy on Test-Set: 16.6% (1657.00003505 / 10000)
Iterations:      0, Training Accuracy:  20.0%
Iterations:    110, Training Accuracy:  92.0%
Iterations:    220, Training Accuracy:  90.0%
Iterations:    330, Training Accuracy:  90.0%
Iterations:    440, Training Accuracy:  88.0%
Iterations:    550, Training Accuracy:  90.0%
Iterations:    660, Training Accuracy: 100.0%
Iterations:    770, Training Accuracy: 100.0%
Iterations:    880, Training Accuracy:  96.0%
Iterations:    990, Training Accuracy:  90.0%
epoca:      2
Accuracy on Test-Set: 97.6% (9764.99974728 / 10000)
Iterations:   1100, Training Accuracy:  98.0%


NameError: name 'mnist' is not defined

### Get and save Models weights and biases

This will save latest trained model params, each of them will be saved as a '\*.npy' file

In [18]:
#get parameters
W,B = net.getWeights()
weights, biases = sess.run([W,B])

layer without params
layer without params


In [19]:
#save parameters on params_path
params_path = path_weights

"""
Save weights to *.npy files, where format is 'typeOfLayer_layerNumber_typeOfParameter.npy', for example
for the first fully-connected layer weights will have 'FC1-W.npy'.

@Ncnn number of convolutional layers with params
@Nfc number of dense layers with params
@weights weights from model
@biases biases from model
"""
def saveWeights(Ncnn, Nfc, weights, biases):
    for i in range(Ncnn):
        np.save(params_path+'CNN'+str(i+1)+'-W.npy', np.array(weights[i]))
        np.save(params_path+'CNN'+str(i+1)+'-B.npy', np.array(biases[i]))

    for i in range(Nfc):
        np.save(params_path+'FC'+str(i+1)+'-W.npy', np.array(weights[i+Ncnn]))
        np.save(params_path+'FC'+str(i+1)+'-B.npy', np.array(biases[i+Ncnn]))

#save model params
saveWeights(2,2,weights,biases)